In [1]:
import gensim
import numpy as np
import json 
caption_file = "captions_200.json"
with open(caption_file, 'r') as f:
    captions = json.load(f)

# len(gensim.utils.simple_preprocess("".join(captions.values()), min_len=4))
total_doc = []
for key in captions:
    text = gensim.utils.simple_preprocess(captions[key])
    total_doc.append(text)


In [2]:
model = gensim.models.Word2Vec(
        window=10,
        min_count=2,
        workers=4
    )
model.build_vocab(total_doc)
model.train(total_doc, total_examples=model.corpus_count, epochs=20)


(10729535, 16991280)

In [3]:
import pickle

with open('text_embedding_200.pkl', 'wb') as file:
    pickle.dump(model, file)

In [35]:
def get_query_vector1(query, model):
    new_query = gensim.utils.simple_preprocess(query)
    query_vector = sum(model.wv[word] for word in new_query) / len(new_query)
    return query_vector

def get_query_vector2(query, model):
    new_query = gensim.utils.simple_preprocess(query)

    query_vector = np.zeros(model.vector_size)
    for word in new_query:
        vec = np.array(model.wv[word])
        vec = vec/np.linalg.norm(vec)
        query_vector += vec

    query_vector =  query_vector / len(new_query)
    return query_vector

similar_words = model.wv.most_similar(positive=[get_query_vector2("sleeveless shirt hat", model)], topn=10)
print(similar_words)

[('sleeveless', 0.5869356989860535), ('shirt', 0.5621093511581421), ('hat', 0.48381856083869934), ('top', 0.28593480587005615), ('medium', 0.2671615183353424), ('short', 0.25188347697257996), ('no', 0.2510964870452881), ('tank', 0.24460361897945404), ('sleeve', 0.24083776772022247), ('with', 0.1875535398721695)]
